In [1]:
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.0.0rc1

In [2]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.0.0rc1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.0.0rc1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

<div class="alert alert-block alert-danger">This notebook is designed for an older StellarGraph version 1.0.0rc1 and may not function correctly with the newer installed version 1.0.0. Please see: <a href="https://github.com/stellargraph/stellargraph/issues/1172">https://github.com/stellargraph/stellargraph/issues/1172</a>.</div>

/home/guomukun/anaconda3/envs/graphemb/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: This notebook is designed for an older StellarGraph version 1.0.0rc1 and may not function correctly with the newer installed version 1.0.0. Please see: <https://github.com/stellargraph/stellargraph/issues/1172>.
  """


In [3]:
import networkx as nx
import pandas as pd
import numpy as np
import os
import random

import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UniformRandomWalk
from stellargraph.data import UnsupervisedSampler
from sklearn.model_selection import train_test_split

from tensorflow import keras
import tensorflow as tf
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score

from stellargraph import globalvar

from stellargraph import datasets
from IPython.display import display, HTML

if tf.test.gpu_device_name():
  print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

### Loading the TF network data

In [25]:
FILE_NAME = 'GM12878_tf2tf.csv'
SEP = ','

df = pd.read_csv('../data/'+FILE_NAME, sep=SEP)

df

,cell_type,source,target,type,weight
0,GM12878,ATF3,ZBTB40,TSS,373.000000
1,GM12878,ATF3,YBX1,TSS,196.371089
2,GM12878,ATF3,BHLHE40,TSS,88.392564
3,GM12878,ATF3,TBP,TSS,144.000000
4,GM12878,ATF3,MXI1,TSS,98.760086
...,...,...,...,...,...
2191,GM12878,JUND,STAT5A,TSS,1000.000000
2192,GM12878,JUND,UBTF,TSS,625.924955
2193,GM12878,JUND,JUNB,TSS,412.366774
2194,GM12878,JUND,JUND,TSS,1000.000000


In [5]:
# Print basic info
nx_graph = nx.from_pandas_edgelist(df[['source', 'target', 'weight', 'cell_type']], 'source', 'target', edge_attr='weight')
print(nx.info(nx_graph))

Name: 
Type: Graph
Number of nodes: 69
Number of edges: 1643
Average degree:  47.6232


In [6]:
G = sg.StellarDiGraph(edges=df[['source', 'target', 'weight']])

print(G.info())

StellarDiGraph: Directed multigraph
 Nodes: 69, Edges: 2196

 Node types:
  default: [69]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [2196]
        Weights: range=[23.9614, 3105.97], mean=671.01, std=536.945


**Use degree as node features**

In [13]:
# node2deg = G.node_degrees()
# features = pd.DataFrame(node2deg.items(), columns=['node', 'features'])
# features.set_index(['node'], drop=True, append=False, inplace=True, verify_integrity=True)
# print(features)
# G = sg.StellarDiGraph(edges=df[['source', 'target', 'weight']], nodes=features)

                 degree
node                   
ZBTB40               87
ATF3                 54
YBX1                 43
BHLHE40              91
TBP                  83
...                 ...
CEBPB                22
ZBED1                47
SUZ12                33
POLR2AphosphoS2      50
POLR2AphosphoS5      57

[69 rows x 1 columns]


**Use 1 as node features|**

In [32]:
node_names = G.nodes()
features = pd.DataFrame(node_names, columns=['node'])
features['features'] = 1
print(features)
features.set_index(['node'], drop=True, append=False, inplace=True, verify_integrity=True)
G = sg.StellarDiGraph(edges=df[['source', 'target', 'weight']], nodes=features)

               node  features
0            ZBTB40         1
1              ATF3         1
2              YBX1         1
3           BHLHE40         1
4               TBP         1
..              ...       ...
64            CEBPB         1
65            ZBED1         1
66            SUZ12         1
67  POLR2AphosphoS2         1
68  POLR2AphosphoS5         1

[69 rows x 2 columns]


**1. Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk, and random seed.**

In [33]:
nodes = list(G.nodes())
number_of_walks = 10
length = 3

**2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.**

In [34]:
unsupervised_samples = UnsupervisedSampler(
    G, nodes=nodes, length=length, number_of_walks=number_of_walks
)

**3. Create a node pair generator:**

Next, create the node pair generator for sampling and streaming the training data to the model. The node pair generator essentially "maps" pairs of nodes `(target, context)` to the input of GraphSAGE: it either takes minibatches of node pairs, or an `UnsupervisedSampler` instance which generates the minibatches of node pairs on demand. The generator samples 2-hop subgraphs with `(target, context)` head nodes extracted from those pairs, and feeds them, together with the corresponding binary labels indicating which pair represent positive or negative sample, to the input layer of the node pair classifier with GraphSAGE node encoder, for SGD updates of the model parameters.

Specify:
1. The minibatch size (number of node pairs per minibatch).
2. The number of epochs for training the model.
3. The sizes of 1- and 2-hop neighbor samples for GraphSAGE:

Note that the length of `num_samples` list defines the number of layers/iterations in the GraphSAGE encoder. In this example, we are defining a 2-layer GraphSAGE encoder.

In [35]:
batch_size = 50
epochs = 20
num_samples = [10, 20, 30] 

In the following we show the working of node pair generator with the UnsupervisedSampler, which will generate samples on demand.

In [36]:
generator = GraphSAGELinkGenerator(G, batch_size, num_samples)
train_gen = generator.flow(unsupervised_samples)

Build the model: a 2-layer GraphSAGE encoder acting as node representation learner, with a link classification layer on concatenated (`citing-paper`, `cited-paper`) node embeddings.

GraphSAGE part of the model, with hidden layer sizes of 50 for both GraphSAGE layers, a bias term, and no dropout. (Dropout can be switched on by specifying a positive dropout rate, 0 < dropout < 1).
**Note that the length of `layer_sizes` list must be equal to the length of `num_samples`, as `len(num_samples)` defines the number of hops (layers) in the GraphSAGE encoder**.

In [37]:
layer_sizes = [50, 50, 50]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=generator, bias=True, dropout=0.0, normalize="l2"
)

In [38]:
# Build the model and expose input and output sockets of graphsage, for node pair inputs:
x_inp, x_out = graphsage.in_out_tensors()

In [39]:
# from pprint import pprint
# pprint(x_inp)

In [40]:
# pprint(x_out)

In [41]:
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [42]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

**4. Train the model.**

In [43]:
history = model.fit(
    train_gen,
    epochs=epochs,
    verbose=1,
    use_multiprocessing=False,
    workers=4,
    shuffle=True,
)

  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 56 steps
Epoch 1/20
56/56 [==============================] - 61s 1s/step - loss: 0.8133 - binary_accuracy: 0.5000
Epoch 2/20
56/56 [==============================] - 70s 1s/step - loss: 0.8119 - binary_accuracy: 0.5000
Epoch 3/20
56/56 [==============================] - 67s 1s/step - loss: 0.8133 - binary_accuracy: 0.5000
Epoch 4/20
56/56 [==============================] - 70s 1s/step - loss: 0.8119 - binary_accuracy: 0.5000
Epoch 5/20
56/56 [==============================] - 68s 1s/step - loss: 0.8105 - binary_accuracy: 0.5000
Epoch 6/20
56/56 [==============================] - 69s 1s/step - loss: 0.8147 - binary_accuracy: 0.5000
Epoch 7/20
56/56 [==============================] - 64s 1s/step - loss: 0.8161 - binary_accuracy: 0.5000
Epoch 8/20
56/56 [==============================] - 70s 1s/step - loss: 0.8119 - binary_accuracy: 0.5000
Epoch 9/20
38/56 [===================>..........] - ETA: 23s - loss: 0.7965 - binary_accuracy: 0.5127

KeyboardInterrupt: 

### Extracting node embeddings
Now that the node pair classifier is trained, we can use its node encoder part as node embeddings evaluator. Below we evaluate node embeddings as activations of the output of graphsage layer stack, and visualise them, coloring nodes by their subject label.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from stellargraph.mapper import GraphSAGENodeGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

**Building a new node-based model**

The `(src, dst)` node pair classifier `model` has two identical node encoders: one for source nodes in the node pairs, the other for destination nodes in the node pairs passed to the model. We can use either of the two identical encoders to evaluate node embeddings. Below we create an embedding model by defining a new Keras model with `x_inp_src` (a list of odd elements in `x_inp`) and `x_out_src` (the 1st element in `x_out`) as input and output, respectively. Note that this model's weights are the same as those of the corresponding node encoder in the previously trained node pair classifier.

In [ ]:
x_inp_src = x_inp[0::2]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

In [ ]:
print(x_inp_src)

We also need a node generator to feed graph nodes to `embedding_model`. We want to evaluate node embeddings for all nodes in the graph:

In [ ]:
node_ids = node_subjects.index
node_gen = GraphSAGENodeGenerator(G, batch_size, num_samples).flow(node_ids)

In [ ]:
node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)

#### Visualize the node embeddings 
Next we visualize the node embeddings in 2D using t-SNE. Colors of the nodes depict their true classes (subject in the case of Cora dataset) of the nodes. 

In [ ]:
node_subject = node_subjects.astype("category").cat.codes

X = node_embeddings
if X.shape[1] > 2:
    transform = TSNE  # PCA

    trans = transform(n_components=2)
    emb_transformed = pd.DataFrame(trans.fit_transform(X), index=node_ids)
    emb_transformed["label"] = node_subject
else:
    emb_transformed = pd.DataFrame(X, index=node_ids)
    emb_transformed = emb_transformed.rename(columns={"0": 0, "1": 1})
    emb_transformed["label"] = node_subject

In [ ]:
alpha = 0.7

fig, ax = plt.subplots(figsize=(7, 7))
ax.scatter(
    emb_transformed[0],
    emb_transformed[1],
    c=emb_transformed["label"].astype("category"),
    cmap="jet",
    alpha=alpha,
)
ax.set(aspect="equal", xlabel="$X_1$", ylabel="$X_2$")
plt.title(
    "{} visualization of GraphSAGE embeddings for cora dataset".format(transform.__name__)
)
plt.show()

In [ ]:
emb_transformed.head()